# Geo API Example

## Setup
Install the Signal Ocean SDK:
```
pip install signal-ocean
```
Set your subscription key acquired here: https://apis.signalocean.com/profile

In [1]:
signal_ocean_api_key = '' #replace with your subscription key

In [2]:
from signal_ocean import Connection
from signal_ocean.geo import GeoAPI
import numpy as np
import pandas as pd
import requests
import json

In [3]:
connection = Connection(signal_ocean_api_key)
api = GeoAPI(connection)

## Areas

### Get all available areas

In [4]:
all_areas = api.get_areas()

In [5]:
df_areas = pd.DataFrame([a.__dict__ for a in all_areas])

In [6]:
df_areas.head(10)

,id,name,area_type_id,parent_area_id
0,2,Arabian Gulf,1,89.0
1,3,Arctic Ocean & Barents Sea,1,25021.0
2,7,Black Sea / Sea Of Marmara,1,93.0
3,9,Caribs,1,25019.0
4,10,East Coast Central America,1,25019.0
5,12,East Coast Canada,1,25019.0
6,13,East Coast Mexico,1,25019.0
7,15,East Coast South America,1,25019.0
8,16,Australia / New Zealand,1,25020.0
9,17,China / Taiwan,1,99.0


### Get a specific area

In [7]:
area_id = 24676

In [8]:
one_area = api.get_area(area_id)

In [9]:
df_AG = pd.DataFrame(one_area.__dict__ , index= [0])

In [10]:
df_AG

,id,name,area_type_id,parent_area_id
0,24676,US Gulf,0,33


### Area usage examples

Explore different area types and how they are related. First, see how many areas of each type exist.

In [11]:
df_areas.area_type_id.value_counts()

4    111
0     54
1     28
2     14
3      4
Name: area_type_id, dtype: int64

View areas of a specific type. Notice how area types 3 and 4 don't have a parent Id

In [12]:
df_areas[df_areas['area_type_id'] == 3]

,id,name,area_type_id,parent_area_id
22,84,East,3,NaN
23,85,Pacific America,3,NaN
209,25027,Africa,3,NaN
210,25028,West,3,NaN


#### Find the parent of an Area

In [13]:
df_areas[df_areas['id'] == df_AG['parent_area_id'].item()]

,id,name,area_type_id,parent_area_id
17,33,US Gulf & Mainland,1,25019.0


In [14]:
df_AG.join(df_areas.set_index('id'), on = 'parent_area_id',lsuffix='_level_0', rsuffix='_level_1')

,id,name_level_0,area_type_id_level_0,parent_area_id_level_0,name_level_1,area_type_id_level_1,parent_area_id_level_1
0,24676,US Gulf,0,33,US Gulf & Mainland,1,25019.0


#### Get a dataframe with all areas of a type and their parents

In [15]:
df_areas[df_areas['area_type_id'] == 2].join(
    df_areas.set_index('id'),
    on = 'parent_area_id',
    lsuffix='_level_0', 
    rsuffix='_level_1').sort_values(by = 'name_level_1')

,id,name_level_0,area_type_id_level_0,parent_area_id_level_0,name_level_1,area_type_id_level_1,parent_area_id_level_1
202,25018,Africa,2,25027.0,Africa,3,NaN
24,89,Arabian Gulf,2,84.0,East,3,NaN
26,99,Far East,2,84.0,East,3,NaN
27,103,Red Sea,2,84.0,East,3,NaN
204,25020,Australia / New Zealand,2,84.0,East,3,NaN
206,25023,India / Pakistan,2,84.0,East,3,NaN
28,111,West Coast Central America,2,85.0,Pacific America,3,NaN
29,112,West Coast Mexico,2,85.0,Pacific America,3,NaN
30,113,West Coast South America,2,85.0,Pacific America,3,NaN
208,25026,West Coast North America,2,85.0,Pacific America,3,NaN


#### Get a dataframe with all area level names

In [16]:
df_0_to_1 = df_areas[df_areas['area_type_id'] == 0].join(df_areas.set_index('id'),
              on = 'parent_area_id', 
              rsuffix='_area_level_1')[['id', 'name', 'name_area_level_1','parent_area_id_area_level_1']]

In [17]:
df_to_2 = df_0_to_1.join(df_areas.set_index('id'),
                        on = 'parent_area_id_area_level_1',
                        rsuffix = '_area_level_2')[['id', 'name', 'name_area_level_1', 'name_area_level_2', 'parent_area_id']]

In [18]:
df_areas_all = df_to_2.join(df_areas.set_index('id'),
             on = 'parent_area_id',
             lsuffix = '_area_level_0',
              rsuffix = '_area_level_3'
            )[['id', 'name_area_level_0', 'name_area_level_1', 'name_area_level_2', 'name_area_level_3']]

In [19]:
df_areas_all.head(10)

,id,name_area_level_0,name_area_level_1,name_area_level_2,name_area_level_3
31,24583,Black Sea,Black Sea / Sea Of Marmara,Black Sea / Sea Of Marmara,West
32,24584,Sea of Marmara,Black Sea / Sea Of Marmara,Black Sea / Sea Of Marmara,West
33,24594,Baltic Sea Upper,Baltic,Baltic / North Sea,West
34,24598,West Coast Central America,West Coast Central America,West Coast Central America,Pacific America
35,24600,East Coast Central America,East Coast Central America,Atlantic America,West
36,24655,Singapore / Malaysia,South East Asia,Far East,East
37,24666,North China,China / Taiwan,Far East,East
38,24671,East Coast Mexico,East Coast Mexico,Atlantic America,West
39,24673,Arctic Ocean & Barents Sea,Arctic Ocean & Barents Sea,Baltic / North Sea,West
40,24676,US Gulf,US Gulf & Mainland,Atlantic America,West


## Countries

### Get available countries

In [20]:
all_countries = api.get_countries()

In [21]:
df_countries = pd.DataFrame([a.__dict__ for a in all_countries])

In [22]:
df_countries.head(10)

,id,name,country_code,country_code_numeric,country_code_iso3
0,3,Azores,1A,999,AZO
1,4,Czechoslovakia,1C,999,CSK
2,5,Madeira,1M,999,PMD
3,6,Neutral Zone (between Saudi Arabia & Iraq),1N,999,NTZ
4,7,Canary Islands,2C,999,CNI
5,8,Andorra,AD,020,AND
6,9,United Arab Emirates,AE,784,ARE
7,10,Afghanistan,AF,004,AFG
8,11,Antigua and Barbuda,AG,028,ATG
9,12,Anguilla,AI,660,AIA


### Get a specific country using its Id

In [23]:
country_id = 9

In [24]:
one_country = api.get_country(country_id)

In [25]:
df_UAA = pd.DataFrame(one_country.__dict__ , index= [0])

In [26]:
df_UAA

,id,name,country_code,country_code_numeric,country_code_iso3
0,9,United Arab Emirates,AE,784,ARE


## Ports

### Get availabe ports

In [27]:
all_ports = api.get_ports()

In [28]:
df_ports = pd.DataFrame([a.__dict__ for a in all_ports])

In [29]:
df_ports.head(10)

,id,country_id,area_id,name,latitude,longitude
0,3153,9,24777,Fujairah,25.1975,56.3839
1,3154,9,24777,Das Island,25.1334,52.92
2,3155,9,24777,Fateh Terminal,25.5771,54.4685
3,3156,9,24777,Mina Saqr,25.9747,55.9403
4,3157,9,24777,Jebel Ali,25.0071,55.0475
5,3158,9,24777,Jebel Dhanna,24.2157,52.6739
6,3159,9,24777,Mubarras Island,24.435,53.5271
7,3160,9,24777,Ruwais,24.1731,52.7152
8,3161,9,24777,Sharjah,25.364,55.3678
9,3162,9,24777,Zirku Island,25.0147,52.9926


### Get a specific port

In [30]:
port_id = 3153

In [31]:
one_port = api.get_port(port_id)

In [32]:
df_Fuj = pd.DataFrame(one_port.__dict__ , index= [0])

In [33]:
df_Fuj

,id,country_id,area_id,name,latitude,longitude
0,3153,9,24777,Fujairah,25.1975,56.3839


## Combined examples

### Get all ports in a specific level 0 area

In [34]:
area_id = 24676

In [35]:
df_Ports_In_Area = df_ports[df_ports['area_id'] == area_id]

In [36]:
df_Ports_In_Area.head(10)

,id,country_id,area_id,name,latitude,longitude
665,3838,240,24676,Baton Rouge,30.4528,-91.1961
668,3841,240,24676,Brownsville,25.9579,-97.3822
672,3845,240,24676,Corpus Christi,27.8096,-97.3595
676,3849,240,24676,Freeport (Texas),28.9413,-95.3186
677,3850,240,24676,Galveston,29.3105,-94.7999
678,3851,240,24676,Garyville,30.0426,-90.6203
680,3853,240,24676,Houston,29.7399,-95.0431
683,3856,240,24676,Lake Charles,30.1934,-93.2759
685,3858,240,24676,Loop,28.8702,-90.0055
688,3861,240,24676,Mobile,30.667,-88.0301


### Get all ports in a specific country

In [37]:
country_id = 9

In [38]:
df_Ports_In_Country = df_ports[df_ports['country_id'] == country_id]

In [39]:
df_Ports_In_Country.head(10)

,id,country_id,area_id,name,latitude,longitude
0,3153,9,24777,Fujairah,25.1975,56.3839
1,3154,9,24777,Das Island,25.1334,52.92
2,3155,9,24777,Fateh Terminal,25.5771,54.4685
3,3156,9,24777,Mina Saqr,25.9747,55.9403
4,3157,9,24777,Jebel Ali,25.0071,55.0475
5,3158,9,24777,Jebel Dhanna,24.2157,52.6739
6,3159,9,24777,Mubarras Island,24.435,53.5271
7,3160,9,24777,Ruwais,24.1731,52.7152
8,3161,9,24777,Sharjah,25.364,55.3678
9,3162,9,24777,Zirku Island,25.0147,52.9926


### Get all areas that a country has ports in

In [40]:
country_id = 240

In [41]:
df_Areas_In_Country = df_areas[df_areas['id'].isin(df_ports[df_ports['country_id'] == country_id]['area_id'])]

In [42]:
df_Areas_In_Country

,id,name,area_type_id,parent_area_id
40,24676,US Gulf,0,33.0
48,24719,Alaska,0,25017.0
54,24732,Great Lakes,0,12.0
55,24733,US Mainland,0,33.0
58,24739,US North Pacific,0,25017.0
60,24741,US West Coast,0,25017.0
63,24747,US Atlantic Coast,0,32.0
83,24782,Pacific Islands,0,20.0


### Joined table

#### Get all the ports and area levels of a country

In [43]:
country_id = 9

In [44]:
df_Ports_incr = df_ports[df_ports['country_id'] == country_id].join(df_countries[['id','name']].set_index('id'),
                                                   on= 'country_id',
                                                   rsuffix = '_country'
                                                                         ).join(df_areas_all.set_index('id'),
                                                                                on = 'area_id') 

In [45]:
df_Ports_incr.head(10)

,id,country_id,area_id,name,latitude,longitude,name_country,name_area_level_0,name_area_level_1,name_area_level_2,name_area_level_3
0,3153,9,24777,Fujairah,25.1975,56.3839,United Arab Emirates,Arabian Gulf,Arabian Gulf,Arabian Gulf,East
1,3154,9,24777,Das Island,25.1334,52.92,United Arab Emirates,Arabian Gulf,Arabian Gulf,Arabian Gulf,East
2,3155,9,24777,Fateh Terminal,25.5771,54.4685,United Arab Emirates,Arabian Gulf,Arabian Gulf,Arabian Gulf,East
3,3156,9,24777,Mina Saqr,25.9747,55.9403,United Arab Emirates,Arabian Gulf,Arabian Gulf,Arabian Gulf,East
4,3157,9,24777,Jebel Ali,25.0071,55.0475,United Arab Emirates,Arabian Gulf,Arabian Gulf,Arabian Gulf,East
5,3158,9,24777,Jebel Dhanna,24.2157,52.6739,United Arab Emirates,Arabian Gulf,Arabian Gulf,Arabian Gulf,East
6,3159,9,24777,Mubarras Island,24.435,53.5271,United Arab Emirates,Arabian Gulf,Arabian Gulf,Arabian Gulf,East
7,3160,9,24777,Ruwais,24.1731,52.7152,United Arab Emirates,Arabian Gulf,Arabian Gulf,Arabian Gulf,East
8,3161,9,24777,Sharjah,25.364,55.3678,United Arab Emirates,Arabian Gulf,Arabian Gulf,Arabian Gulf,East
9,3162,9,24777,Zirku Island,25.0147,52.9926,United Arab Emirates,Arabian Gulf,Arabian Gulf,Arabian Gulf,East


#### Use the above example for all countries to find the top Countries with the most Ports

In [46]:
df_Ports_country_count = df_Ports_incr = df_ports.join(df_countries[['id','name']].set_index('id'),
                                                   on= 'country_id',
                                                   rsuffix = '_country'
                                                                         ).join(df_areas_all.set_index('id'),
                                                                                on = 'area_id') 

In [47]:
df_Ports_country_count.name_country.value_counts().head(10)

United States     195
Japan             158
Indonesia         142
China             116
Canada             89
United Kingdom     85
Australia          75
Norway             72
Brazil             58
Philippines        57
Name: name_country, dtype: int64